In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("EvidenceCatalog.csv")
#data.head()

In [4]:
data.columns

Index(['NAME', 'CREATED_BY', 'DATE_CREATED', 'UPDATED_BY', 'DATE_UPDATED',
       'TITLE', 'STUDYTYPE', 'SPONSORINSTITUTION', 'SPONSORINGDIVISION',
       'SPONSORINGUNIT', 'CATEGORY', 'INDICATION', 'PRIMARYDRUG',
       'PFECOMPOUND', 'STATUS', 'PASS', 'PMS', 'HTASUBMISSION',
       'REGULATORYSUBMISSION', 'COUNTRIESOFSTUDY', 'UNITEDSTATES',
       'INTERNATIONALPRIORITY', 'ANCHORMARKET', 'RISKSTUDY', 'RISKTYPE',
       'RISKDESCRIPTION', 'RISKMITIGATIONPLAN', 'EXECUTIONGROUP',
       'EVIDENCELEAD', 'STUDYOPSLEAD', 'CLINICALSCIENTISTRWE', 'RWESCIENTIST',
       'RWESTRATEGIST', 'QUALITYANDCOMPLAINCE', 'STATISTICIAN',
       'MEDICALAFFAIRS', 'EVIDENCESOURCE', 'TRANSITIONPLAN', 'STUDYSOP',
       'PRIMARYDATACOLLECTION', 'SECONDARYDATACOLLECTION', 'STUDYSUBTYPE',
       'COMPLETIONDATE', 'FINALREPORTDATE', 'FSFV', 'LSLV'],
      dtype='object')

In [5]:
df = data[['STATUS' , 'STUDYSOP','COMPLETIONDATE', 'FINALREPORTDATE', 'FSFV', 'LSLV']]

In [6]:
date_fields = ['COMPLETIONDATE', 'FINALREPORTDATE', 'FSFV', 'LSLV']

In [7]:
def calculate_missingness(group):
    print(group.head())
    total_rows = len(group)
    missingness = {
        "STUDYSOP": group['STUDYSOP'].iloc[0],
        "StudyStatus":group['STATUS'].iloc[0],
    }
    for field in date_fields:
        missing_count = group[field].isna().sum()
        missingness[field] =(missing_count/len(group[field]))*100
    return pd.DataFrame([missingness])

In [8]:
df.head()

,STATUS,STUDYSOP,COMPLETIONDATE,FINALREPORTDATE,FSFV,LSLV
0,"Approved, Closed",GNT01,4/30/2025,NaN,NaN,NaN
1,COMPLETED,CT02,NaN,NaN,5/1/1985,NaN
2,"Rejected, Request",GNT01,10/9/2026,NaN,NaN,NaN
3,COMPLETED,CT02,NaN,4/3/1992,NaN,NaN
4,TERMINATED,CT02,NaN,NaN,2/1/1993,NaN


In [9]:
reshaped_df = df.melt(
    id_vars = ['STUDYSOP','STATUS'],
    value_vars = date_fields,
    var_name = 'DateField',
    value_name = 'DateValue'
)

In [10]:
reshaped_df.head()

,STUDYSOP,STATUS,DateField,DateValue
0,GNT01,"Approved, Closed",COMPLETIONDATE,4/30/2025
1,CT02,COMPLETED,COMPLETIONDATE,NaN
2,GNT01,"Rejected, Request",COMPLETIONDATE,10/9/2026
3,CT02,COMPLETED,COMPLETIONDATE,NaN
4,CT02,TERMINATED,COMPLETIONDATE,NaN


In [11]:
grouped = reshaped_df.groupby(['STUDYSOP','STATUS','DateField'])

In [12]:
grouped.head()

,STUDYSOP,STATUS,DateField,DateValue
0,GNT01,"Approved, Closed",COMPLETIONDATE,4/30/2025
1,CT02,COMPLETED,COMPLETIONDATE,NaN
2,GNT01,"Rejected, Request",COMPLETIONDATE,10/9/2026
3,CT02,COMPLETED,COMPLETIONDATE,NaN
4,CT02,TERMINATED,COMPLETIONDATE,NaN
...,...,...,...,...
160961,CT45,CANCELLED,LSLV,NaN
161906,RC01,"Canceled, Concept Approved",LSLV,NaN
162932,RC01,"Canceled, Concept Approved",LSLV,NaN
163657,GNT01,External,LSLV,NaN


In [43]:
result_df = grouped.apply(lambda g:pd.Series({
    "PopulatedCount":(g['DateValue'].notna().sum()),
    "% Populated":(g['DateValue'].notna().sum()/len(g))*100,
    "MissinnessCount":(g['DateValue'].isna().sum()),
    "% Missingness":(g["DateValue"].isna().sum()/len(g))*100
})).reset_index()

C:\Users\kadurr\AppData\Local\Temp\ipykernel_11184\2911073311.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = grouped.apply(lambda g:pd.Series({


In [44]:
result_df['STATUS']= result_df['STATUS'].fillna('Unknown')

In [45]:
result_df.head()

,STUDYSOP,STATUS,DateField,PopulatedCount,% Populated,MissinnessCount,% Missingness
0,CT02,APPROVED,COMPLETIONDATE,0.0,0.000000,37.0,100.000000
1,CT02,APPROVED,FINALREPORTDATE,0.0,0.000000,37.0,100.000000
2,CT02,APPROVED,FSFV,0.0,0.000000,37.0,100.000000
3,CT02,APPROVED,LSLV,0.0,0.000000,37.0,100.000000
4,CT02,CANCELLED,COMPLETIONDATE,3.0,0.033829,8865.0,99.966171


In [46]:
#result_df.to_csv("Missingness_date.csv")

In [91]:
def normalize_str(a_string: str):
    return str(a_string).strip().title()

In [48]:
status_ah = pd.read_csv('StatusHarmonization.csv')
status_ah.head()

,status_native,sop,source,status_description,harmonized_status,harmonized_status_detail,review
0,APPROVED,"CT24, CT45",GDPIP,The Final Approved Protocol (FAP) contains all...,Approved,No additional details,NaN
1,CANCELLED,"CT24, CT45",GDPIP,Study has been cancelled prior to any subjects...,Canceled,No additional details,NaN
2,COMPLETED,"CT24, CT45",GDPIP,Last Subject Last Visit (LSLV) has occurred (e...,Completed,No additional details,NaN
3,CONCEPT,"CT24, CT45",GDPIP,NaN,Concept,No additional details,NaN
4,NOT PURSUED,"CT24, CT45",GDPIP,Old status,Not Pursued,No additional details,NaN


In [49]:
status_ah.status_native = status_ah.status_native.apply(lambda x: normalize_str(x))
status_ah.harmonized_status = status_ah.harmonized_status.apply(lambda x: normalize_str(x))
status_ah = status_ah[['status_native', 'harmonized_status']].drop_duplicates()
status_ah.head()

,status_native,harmonized_status
0,Approved,Approved
1,Cancelled,Canceled
2,Completed,Completed
3,Concept,Concept
4,Not Pursued,Not Pursued


In [50]:
result_df['STATUS'] = result_df['STATUS'].fillna('Unknown') 
result_df['STATUS'] = result_df['STATUS'].apply(lambda x: normalize_str(x))


In [51]:
result_df.head()

,STUDYSOP,STATUS,DateField,PopulatedCount,% Populated,MissinnessCount,% Missingness
0,CT02,Approved,COMPLETIONDATE,0.0,0.000000,37.0,100.000000
1,CT02,Approved,FINALREPORTDATE,0.0,0.000000,37.0,100.000000
2,CT02,Approved,FSFV,0.0,0.000000,37.0,100.000000
3,CT02,Approved,LSLV,0.0,0.000000,37.0,100.000000
4,CT02,Cancelled,COMPLETIONDATE,3.0,0.033829,8865.0,99.966171


In [52]:
result_df = pd.merge(result_df, status_ah, left_on='STATUS', right_on='status_native', how='left')
result_df = result_df.drop(['status_native', 'STATUS'], axis=1).rename(columns={'harmonized_status': 'STATUS'})

In [53]:
result_df.head()

,STUDYSOP,DateField,PopulatedCount,% Populated,MissinnessCount,% Missingness,STATUS
0,CT02,COMPLETIONDATE,0.0,0.000000,37.0,100.000000,Approved
1,CT02,FINALREPORTDATE,0.0,0.000000,37.0,100.000000,Approved
2,CT02,FSFV,0.0,0.000000,37.0,100.000000,Approved
3,CT02,LSLV,0.0,0.000000,37.0,100.000000,Approved
4,CT02,COMPLETIONDATE,3.0,0.033829,8865.0,99.966171,Canceled


In [54]:
result_df.columns

Index(['STUDYSOP', 'DateField', 'PopulatedCount', '% Populated',
       'MissinnessCount', '% Missingness', 'STATUS'],
      dtype='object')

In [55]:
result_df = result_df[['STUDYSOP','STATUS', 'DateField', "PopulatedCount",'% Populated', "MissinnessCount", '% Missingness']]

In [56]:
result_df['STATUS']= result_df['STATUS'].fillna('Unknown')

In [57]:
result_df

,STUDYSOP,STATUS,DateField,PopulatedCount,% Populated,MissinnessCount,% Missingness
0,CT02,Approved,COMPLETIONDATE,0.0,0.000000,37.0,100.000000
1,CT02,Approved,FINALREPORTDATE,0.0,0.000000,37.0,100.000000
2,CT02,Approved,FSFV,0.0,0.000000,37.0,100.000000
3,CT02,Approved,LSLV,0.0,0.000000,37.0,100.000000
4,CT02,Canceled,COMPLETIONDATE,3.0,0.033829,8865.0,99.966171
...,...,...,...,...,...,...,...
335,RC01,Completed,LSLV,0.0,0.000000,1.0,100.000000
336,RC01,Unknown,COMPLETIONDATE,1.0,100.000000,0.0,0.000000
337,RC01,Unknown,FINALREPORTDATE,0.0,0.000000,1.0,100.000000
338,RC01,Unknown,FSFV,0.0,0.000000,1.0,100.000000


In [58]:
result_df.to_csv("Missingness_harmonization.csv")

In [59]:
import pandas as pd 

In [82]:
data = pd.read_csv("EvidenceCatalog.csv")

In [83]:
df = data[['NAME','COUNTRIESOFSTUDY']]

In [84]:
df['COUNTRIESOFSTUDY']=df['COUNTRIESOFSTUDY'].str.split('|')

C:\Users\kadurr\AppData\Local\Temp\ipykernel_11184\1554785923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['COUNTRIESOFSTUDY']=df['COUNTRIESOFSTUDY'].str.split('|')


In [86]:
df = df.explode('COUNTRIESOFSTUDY').reset_index(drop =True)

In [87]:
df['COUNTRIESOFSTUDY']=df['COUNTRIESOFSTUDY'].str.split(',')
df_country = df.explode('COUNTRIESOFSTUDY').reset_index(drop =True)

In [88]:
df_country.rename(columns ={'NAME':'STUDYID','COUNTRIESOFSTUDY':'COUNTRY'},inplace=True)

In [89]:
df_country

,STUDYID,COUNTRY
0,53234877,UNITED STATES
1,PRXWAN85014,NIGERIA
2,PRXWAN85014,UNKNOWN
3,90671761,BRAZIL
4,A112232,NEW ZEALAND
...,...,...
84640,53232265,HUNGARY
84641,1032006,UNITED STATES
84642,78431121,ITALY
84643,86864321,PUERTO RICO


In [92]:
df_country.STUDYID = df_country.STUDYID.apply(lambda x: normalize_str(x))

In [93]:
df_country.to_csv('Countries_data2.csv')